# Question 1
a) Dual problem

min $15p_1 + 15p_2 + 15p_3$

s.t. $p_1, p_2, p_3 \leq 0$

$p_1 + 2p_2 + 2p_3 \leq 1$

$2p_1 + p_2 + 2p_3 \leq 1$

$2p_1 + 2p_2 + p_3 \leq 1$

b) Show that optimal solution is $x_1 = x_2 = x_3 = 3$

We show the optimal solution by solving the primal problem

In [10]:
from typing import Dict
from gurobipy import *
import numpy as np
def qn1():
    # Preparing an optimization model
    model = Model("primal")

    # Declaring variables
    x = model.addMVar(3, name='x')

    print(x)
    # Adding constraints
    constraints_mat = np.array([[1, 2, 2],
                                [2, 1, 2],
                                [2, 2,1]])

    model.addConstr(constraints_mat @ x <= np.array([15, 15 ,15]), name="constraints")

    model.setObjective(np.array([1,1,1]) @ x, GRB.MAXIMIZE)


    model.optimize()
    # print optimal solutions
    print("Primal Solutions:")
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))


    print("Dual Solutions:")
    for d in model.getConstrs():
        print('%s %g' % (d.ConstrName, d.Pi))

qn1()

<(3,) matrix variable>
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0xb6e83bc2
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+30   7.500000e+30   3.000000e+00      0s
       3    9.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  9.000000000e+00
Primal Solutions:
x[0] 3
x[1] 3
x[2] 3
Dual Solutions:
constraints[0] 0.2
constraints[1] 0.2
constraints[2] 0.2


(c) What are the shadow prices associated with the first three constraints.

The shadow prices are $p_1, p_2, p_3 = 0.2$


# Question 2

a) Formulate a linear program to maximum the daily profit of the company
Decision variables:

$N_i$: The number of product $i$ to production, where i = product one and two.

Constants:

$L_i$: The labour cost for one unit of product $i$

$T_i$: The testing hours for one unit of product $i$


Constraints:
$\sum_{i\in products}L_iN_i \leq 90$

$\sum_{i\in products}T_iN_i \leq 90$

$L_1 \geq 200$

$L_2 \geq 100$

Objective function:
max $(7-3)N_1 + (9-1)N_2$ =
max $4N_1 + 8N_2$

b) Provide the python code for the shadow prices of assembly labor and testing

In [ ]:
from gurobipy import *
import numpy as np

def qn2():


    # Preparing an optimization model
    model = Model("qn2")

    # Declaring variables
    N_1 = model.addVar(name='N_1')
    N_2 = model.addVar(name='N_2')

    # Declare constants
    L_1 = 1/5
    L_2 = 3/10
    T_1 = 1/7
    T_2 = 3/7


    # Adding constraints
    model.addConstr(L_1 * N_1 + L_2 * N_2 <= 90, name="Labour constraints")
    model.addConstr(T_1 * N_1 + T_2 * N_2 <= 90, name="Testing constraints")

    model.addConstr(N_1 >= 200, name="Item number 1 constraints")
    model.addConstr(N_2 >= 100, name="Item number 2 constraints")

    model.setObjective(4* N_1 + 8*N_2, GRB.MAXIMIZE)


    model.optimize()
    # print optimal solutions
    print("Primal Solutions:")
    for v in model.getVars():
        print('%s %g' % (v.varName, v.x))


    print("Dual Solutions:")
    for d in model.getConstrs():
        print('%s %g' % (d.ConstrName, d.Pi))

qn2()

Qn 2b answer: Shadow price for labour is 13.33, testing is 9.33

## Qn 3
Formulate the LOP and solve for an assignment that minimizes total race time

Decision variables:
$S_{ij}$ Whether or not swimmer i should swim for style j

Constants: $T_{ij}$ Time for swimmer i to swim for style j

Objective function: min $\sum_i\sum_j T_{ij} S_{ij}$

Constraints:

$\sum_{i\in{swimmers}} S_{ij} = 1$ for all j $\in$ styles. This indicates there is only one swimmer for each style.
$\sum_{j\in{styles}} S_{ij} = 1$ for all i $\in$ swimmers. This indicates there is only each swimmer must swim at least one syle.

$Sij$ are all binary


In [38]:
from gurobipy import *
import numpy as np
def qn3():
    # Preparing an optimization model
    model = Model("qn3")

    # Declaring variables
    S = model.addMVar((4,4), name='S', vtype=GRB.BINARY)

    # Adding style matrix
    # Use float('inf') to represent N.A
    style_mat = np.array([[48.3, 9999999, 66.0, 55.4],
                          [50.2, 53.2, 58.9, 56.2],
                          [47.5, 49.8, 9999999, 53.0],
                          [48.1, 51.8, 64.4, 54.7]])

    for style in [0, 1,2,3]:
        model.addConstr(S[:,style].sum() == 1, name="style constraints")
    for swimmer in [0, 1,2,3]:
        model.addConstr(S[swimmer,:].sum() == 1, name="swimmer constraints")


    # Gurobi can't do 4x4 mul 4x4 matrix with MVars, so we need to do this instead
    obj = sum(S[swimmer,:] @ style_mat[swimmer,:] for swimmer in [0,1,2,3])
    model.setObjective(obj, GRB.MINIMIZE)


    model.optimize()
    # print optimal solutions
    print("Primal Solutions:")

    results = []
    for v in model.getVars():
        results.append(v.x)
        # print('%s %g' % (v.varName, v.x))

    print("Results")
    print(np.array(results).reshape((4,4)))
qn3()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0xccf4dcdb
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1.000016e+07
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 2.117000e+02, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     211.7000000  211.70000  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.01 seconds
Thread count was 12 (of 12 available processors)

Qn 3 Answer:
 Swimmer 1 should swim for FreeStyle.
Swimmer 2 should swim for BreastStroke.
Swimmer 3 should swim for Butterfly.
Swimmer 4 should swim for Backstroke

This gives a total time of 211.7 seconds

## Qn 4
a) Maximize the total flow from Node 1 to Node 9


Constraints:
Flow balance constraints

Non negativity


Objective function:
max $flow$

Constants:

$C_{ij}$: Capacity of arc from Node i to j

Decision variables:
$X_{ij}$: Amount to ship from one node $i$ to node $j$. $X_{12}$, $X_{13}$, $X_{15}$,
$X_{23}$, $X_{26}$, $X_{34}$, $X_{47}$, $X_{49}$, $X_{56}$, $X_{57}$, $X_{69}$, $X_{79}$,

Flow Node Constraints:

Node 1 (source): $flow$ = $X_{12}$ + $X_{13}$ + $X_{15}$

Node 9 (sink) :$flow$ = $X_{69}$ + $X_{79}$ + $X_{49}$

Other nodes:
$\sum_{a\in{nodes}} X_{aj}$ = $\sum_{b\in{nodes}}X_{jb}$ for all j = 2 ... 8


Capacity Constraints $0 \leq X_{ij} \leq C_{ij}$:

$X_{12} \leq 7$

$X_{13} \leq 7$

$X_{15} \leq 8$

$X_{23} \leq 5$

$X_{26} \leq 10$

$X_{26} \leq 10$

$X_{34} \leq 12$

$X_{47} \leq 4$

$X_{49} \leq 7$

$X_{56} \leq 11$

$X_{57} \leq 12$

$X_{69} \leq 13$

$X_{79} \leq 2$


In [103]:
from gurobipy import *
import numpy as np
def qn3():
    # Preparing an optimization model
    model = Model("qn3")


    arcs, arcs_caps_dict = multidict({(1,2): 9, (1,3): 5, (1,5): 20, (2,3): 18, (2,6): 20, (2,7): 15,
                       (3,4): 15, (4,7): 5, (4,9): 16, (5,6): 14, (5,7): 5, (6,9): 6,
                       (7,9): 8
                       })

    variables = model.addVars(arcs, ub = arcs_caps_dict)
    nodes = range(1,10)

    for a in arcs.select('*', 2):
        print(a)
    for a in (arcs.select(2, '*')):
        print(a)


    # Add flow constraints
    for node in nodes[2:9]: # Don't include first and last nodes!
        lhs = [variables[a] for a in arcs.select('*', node)]
        rhs = [variables[b] for b in arcs.select(node, '*')]
        model.addConstr(sum(lhs) == sum(rhs))

    flow = sum(variables[a] for a in arcs.select(1,'*'))
    model.addConstr(flow == sum(variables[a] for a in arcs.select('*', 9))) # All nodes that end with 9.

    # Add capacity constraints
    model.setObjective(flow, GRB.MAXIMIZE)


    model.optimize()
    # print optimal solutions
    print("Solutions for each arc:")

    #printing the optimal solutions obtained
    p = model.getAttr('x', variables)
    print("For Max Total Flow:")
    for arc in arcs:
        print("Flow on Link %s: %g" % (arc , p[arc]))

qn3()

(1, 2)
(2, 3)
(2, 6)
(2, 7)
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 8 rows, 13 columns and 25 nonzeros
Model fingerprint: 0xb2e3f293
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 2e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 8 rows and 13 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective -0.000000000e+00
Solutions for each arc:
For Max Total Flow:
Flow on Link (1, 2): 0
Flow on Link (1, 3): 0
Flow on Link (1, 5): 0
Flow on Link (2, 3): 0
Flow on Link (2, 6): 0
Flow on Link (2, 7): 0
Flow on Link (3, 4): 0
Flow on Link (4, 7): 0
Flow on Link (4, 9): 0
Flow on Link (5, 6): 0
Flow on Link (5

In [ ]:
def qn3b():
    model = Model('qn3b')
    nodes = range(1,10)
    intermediate_nodes = nodes[2:9]
    
    arcs, arcs_cost_dict = multidict({(1,2): 9, (1,3): 5, (1,5): 20, (2,3): 18, (2,6): 20, (2,7): 15,
                       (3,4): 15, (4,7): 5, (4,9): 16, (5,6): 14, (5,7): 5, (6,9): 6,
                       (7,9): 8
                       })

    variables = model.addVars(arcs)

    #defining decision variables
    
    #setting the objective
    model.setObjective(quicksum(variables[arc]*arcs_cost_dict[arc] for arc in arcs), GRB.MINIMIZE)

    #adding the flow conservation constraints (outflow - inflow)
    model.addConstrs(quicksum(variables[arc] for arc in arcs.select(i,'*')) ==  quicksum(variables[arc] for arc in arcs.select('*',i))
                     for i in intermediate_nodes)
    model.addConstr(quicksum(variables[arc] for arc in arcs.select(1,'*')) - quicksum(variables[arc] for arc in arcs.select('*',1))
                    == 1)
    model.addConstr(quicksum(variables[arc] for arc in arcs.select(9,'*')) - quicksum(variables[arc] for arc in arcs.select('*',9))
                    == -1)

    model.optimize()

    #printing the optimal solutions obtained
    print("\nOptimal Value: \t%g\n" % model.objVal)

    #printing the optimal solutions obtained
    p = model.getAttr('x', variables)
    print("Shortest Path:")
    for arc in arcs:
        if p[arc] == 1:
            print("Move from Node %g to Node %g" % (link[0] , link[1]))

qn3b()